### Ingesta de datos del proyecto PI_ML_OPS cohorte DTS-12 
#### https://github.com/Raftxo-MX/PI_ML_OPS.git

Importamos las librerías necesarias en el proceso ETL


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

Cargamos los datos CSV en dataframes:

In [3]:
movies_df = pd.read_csv('./data/movies_dataset.csv')
credits_df = pd.read_csv('./data/credits.csv')

C:\Users\raftx\AppData\Local\Temp\ipykernel_17864\3250179558.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('./data/movies_dataset.csv')


In [6]:
# Al retornarnos un DtypeWarning, intentamos ingestar movies_df
# indicando qué tipo queremos para cada columna

# La idea era buena, pero no funciona realmente, da ValueError cuando hay datos que le
# indico que sean int y son strings
movies_df = pd.read_csv('./data/movies_dataset.csv',
                        dtype={'adult':'str',
                               'belongs_to_collection':'str',
                               'budget':'int',
                               'genres':'str',
                               'homepage':'str',
                               'id':'int',
                               'imdb_id':'str',
                               'original_language':'str',
                               'original_title':'str',
                               'overview':'str',
                               'popularity':'str',
                               'poster_path':'str',
                               'production_companies':'str',
                               'production_countries':'str',
                               'release_date':'datetime64[D]',
                               'revenue':'int',
                               'runtime':'int',
                               'spoken_languages':'str',
                               'status':'str',
                               'tagline':'str',
                               'title':'str',
                               'video':'str',
                               'vote_average':'float64',
                               'vote_count':'int',                               
                               },
                               low_memory=False)

ValueError: invalid literal for int() with base 10: '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'

In [ ]:
movies_df['budget'] = pd.to_numeric(movies_df['budget'], 
                                    errors='coerce').fillna(0).astype('int')

In [7]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [1]:
import pandas as pd
import numpy as np
import re # regular expressions para normalizar algunas columnas
# para las funciones de la API:
import json
from json.decoder import JSONDecodeError 


In [ ]:
# Lee los datos
df_cc = pd.read_csv('./data/credits.csv') 
df_movies = pd.read_excel('./data/movies_dataset.xlsx')

# Une los datasets en un DataFrame llamado 'df'
# Usando la columna 'id' para unirlos 
df = pd.merge(df_cc, df_movies, on='id')

In [ ]:
# Un pequeño estudio de los datos:
# La dimensión:
df.shape

In [5]:
# Las columnas que contiene:
df.columns

Index(['cast', 'crew', 'id', 'adult', 'belongs_to_collection', 'budget',
       'genres', 'homepage', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
# Los valores nulos en cada columna:
df.isnull().sum()

cast                         0
crew                         0
id                           0
adult                        0
belongs_to_collection    41038
budget                       0
genres                       0
homepage                 37746
imdb_id                     17
original_language           11
original_title               0
overview                   959
popularity                   3
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      3
runtime                    260
spoken_languages             3
status                      84
tagline                  25100
title                        3
video                        3
vote_average                 3
vote_count                   3
dtype: int64

___
TRANSFORMACIONES

In [7]:
# Voy a empezar las transformaciones propuesta por el final, quitando las columnas
quitar_columnas = ['video','imdb_id','adult','original_title','poster_path','homepage']
# drop = quitar, axis=1 = columnas (eje 0 filas, eje 1 columnas), inplace=True modificar el dataframe existente en lugar de devolver una copia
df.drop(quitar_columnas,axis=1,inplace=True)

In [14]:
# Compruebo las columnas ahora:
df.shape


(45538, 20)

In [13]:
# antes (45538, 26) / después (45538, 20)
df.columns

Index(['crew', 'id', 'belongs_to_collection', 'budget', 'genres',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'director_names'],
      dtype='object')

In [10]:
# En el desarrollo API posterior solo necesitaremos el nombre del director de cada película
# Todos los demás datos de la columna 'crew' y toda la columna entera de 'cast' SOBRAN.
# Elimino la columna 'cast'
df.drop('cast', axis=1, inplace=True)
df.columns

Index(['crew', 'id', 'belongs_to_collection', 'budget', 'genres',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [12]:
import ast

# tras mucha investigación al final acabo con la biblioteca "ast"
# para convertir los valores 'string' de la columna 'crew' en lista de diccionarios.
df['crew'] = df['crew'].apply(ast.literal_eval)


# Seguidamente aplico la funcion lambda a todos los valores de 'crew' para extraer nombres de los directores
# compruebo con isinstance(x, list) que no haya valores None u otros que no sean de lista.
df['director_names'] = df['crew'].apply(lambda x: [d['name'] for d in x if d['job'] == 'Director'] if isinstance(x, list) else None)

___

In [67]:
matrix_row = df[df['title'] == 'The Congress']
print(matrix_row) 

           id belongs_to_collection budget  \
19999  152795                            0   

                                    genres original_language  \
19999  [Drama, Science Fiction, Animation]                en   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [19]:
df['director_names'].iloc[15:30]

15                                    [Martin Scorsese]
16                                            [Ang Lee]
17    [Allison Anders, Alexandre Rockwell, Robert Ro...
18                                     [Steve Oedekerk]
19                                       [Joseph Ruben]
20                                   [Barry Sonnenfeld]
21                                          [Jon Amiel]
22                                     [Richard Donner]
23                                       [Victor Salva]
24                                        [Mike Figgis]
25                                      [Oliver Parker]
26                                [Lesli Linka Glatter]
27                                      [Roger Michell]
28                      [Jean-Pierre Jeunet, Marc Caro]
29                                        [Zhang Yimou]
Name: director_names, dtype: object

In [68]:
pd.set_option('display.max_colwidth', None)
print(df['director_names'].iloc[17])

['Allison Anders', 'Alexandre Rockwell', 'Robert Rodriguez', 'Quentin Tarantino']


In [21]:
df.columns
# ya tenemos los nombres de los directores en la columna 'director_names' así que me voy a deshacer de la columna 'crew'
df.drop('crew',axis=1,inplace=True)

In [22]:
pd.set_option('display.max_colwidth', None)  # Set the maximum column width to XX characters or None for no limitations.
df['belongs_to_collection'].iloc[128:145]

128                                                                                                                                                     NaN
129                                                                                                                                                     NaN
130                                                                                                                                                     NaN
131                                                                                                                                                     NaN
132                                                                                                                                                     NaN
133                                                                                                                                                     NaN
134                                                             

___
# BACKUP code

In [ ]:
# Antes de continuar voy a hacer una copia de seguridad del dataframe tal como está.
# Guardar copia de seguridad 
# df.to_pickle('./data/df_backup.pickle')

# ...

# Restaurar desde copia de seguridad  
df = pd.read_pickle('./data/df_backup.pickle')

# end of BACKUP code
___

In [23]:
# Vamos a normalizar la columna 'belongs_to_collection'

# la función normalize_collection está preparada especificamente para dicha columna, ya que sabemos que contiene valores NaN
# o unos chorizos con el nombre 'name' de la colección o franquicia.
# Se usa "regular expressions" (re - para los amigos)

def normalize_collection(value):
    if pd.isnull(value):
        return ''
    elif isinstance(value, str):
        matches = re.findall(r"'name': '(.+?)'", value)
        if matches:
            name = matches[0]
            name = name.replace(' Collection', '')
            return name
        else:
            return ''
    else:
        return value['name']
    
df['belongs_to_collection'] = df['belongs_to_collection'].apply(normalize_collection)

In [24]:
# comprobamos el resultado de la normalización:
df['belongs_to_collection'].iloc[18:44]

18      Ace Ventura
19                 
20     Chili Palmer
21                 
22                 
23                 
24                 
25                 
26                 
27                 
28                 
29                 
30                 
31                 
32                 
33             Babe
34                 
35                 
36                 
37                 
38                 
39                 
40                 
41                 
42                 
43    Mortal Kombat
Name: belongs_to_collection, dtype: object

___

In [14]:
# por si acaso...
# df.to_pickle('./data/df_backup2.pickle')

# ...

# Restaurar desde copia de seguridad  
df = pd.read_pickle('./data/df_backup2.pickle')

___

In [16]:
# funcion para desanidar las columnas faltantes
def extract_nested_data(names):
    """
    Extrae los nombres de los objetos dentro de una estructura de datos anidados
    que se proporciona como una cadena de texto en formato de diccionario de Python.

    Parámetros:
    names (str): La cadena de texto que contiene la estructura de datos anidados en formato de diccionario de Python.

    Retorna:
    list: Una lista que contiene los nombres de los objetos dentro de la estructura de datos anidados.

    Ejemplo:
    extract_nested_data('[{"id": 1, "name": "Roberto"}, {"id": 2, "name": "Emilio"}]')
    ['Roberto', 'Emilio']
    """
    try:
        object_list = eval(names)
        object_names = [object_dict['name'] for object_dict in object_list]
        return object_names
    except:
        return ''

In [17]:
df['production_companies'] = df['production_companies'].apply(extract_nested_data)

In [19]:
df['production_countries'] = df['production_countries'].apply(extract_nested_data)

In [22]:
df['genres'] = df['genres'].apply(extract_nested_data)

In [27]:
df['spoken_languages'] = df['spoken_languages'].apply(extract_nested_data)

___
2023-07-05 18:00 RAFTXO marcapáginas, hasta aquí, en teoría todo bien...

In [106]:
# punto de backup. Ahora mismo tenemos Production Companies, Production Countries,
# Genres y Spoken Languagues OK como listas

# df.to_pickle('./data/df_backup3.pickle')
# 
df = pd.read_pickle('./data/df_backup3.pickle')

In [107]:
df.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'director_names'],
      dtype='object')

In [108]:
# comprobaciones
df.iloc[100:102]
df.shape

(45538, 19)

___

In [109]:
# Creamos una función para comprobar duplicados
def check_duplicates(df, column):
    # Encuentra duplicados en la columna especificada
    duplicate_values = df[df.duplicated(column)]
    
    # Contar frecuencia de cada valor duplicado
    duplicate_counts = duplicate_values[column].value_counts()
    
    # Extraer lista de valores duplicados
    duplicate_list = duplicate_counts.index.tolist()
    
    # Cantidad total de valores duplicados
    total_duplicates = len(duplicate_list)
    
    print(f"Total Duplicates in {column}: {total_duplicates}")
    
    # Imprimir cada valor duplicado y su frecuencia
    for value in duplicate_list:
        count = duplicate_counts[value]
        print(f"{value} occurs {count} times")

In [110]:
check_duplicates(df, 'id')

Total Duplicates in id: 43
141971 occurs 8 times
105045 occurs 3 times
13209 occurs 3 times
265189 occurs 3 times
152795 occurs 3 times
119916 occurs 3 times
25541 occurs 3 times
99080 occurs 3 times
42495 occurs 3 times
132641 occurs 3 times
18440 occurs 3 times
97995 occurs 3 times
168538 occurs 3 times
159849 occurs 3 times
11115 occurs 3 times
77221 occurs 3 times
15028 occurs 3 times
14788 occurs 3 times
22649 occurs 3 times
84198 occurs 3 times
10991 occurs 3 times
110428 occurs 3 times
298721 occurs 3 times
12600 occurs 3 times
4912 occurs 3 times
5511 occurs 3 times
23305 occurs 3 times
69234 occurs 3 times
109962 occurs 3 times
157301 occurs 1 times
187156 occurs 1 times
43629 occurs 1 times
8767 occurs 1 times
123634 occurs 1 times
9755 occurs 1 times
24026 occurs 1 times
142563 occurs 1 times
11752 occurs 1 times
24023 occurs 1 times
199591 occurs 1 times
125458 occurs 1 times
116723 occurs 1 times
3057 occurs 1 times


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45535 entries, 0 to 45537
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     45535 non-null  object 
 1   belongs_to_collection  45535 non-null  object 
 2   budget                 45535 non-null  object 
 3   genres                 45535 non-null  object 
 4   original_language      45524 non-null  object 
 5   overview               44576 non-null  object 
 6   popularity             45535 non-null  object 
 7   production_companies   45535 non-null  object 
 8   production_countries   45535 non-null  object 
 9   release_date           45451 non-null  object 
 10  revenue                45535 non-null  float64
 11  runtime                45278 non-null  float64
 12  spoken_languages       45535 non-null  object 
 13  status                 45454 non-null  object 
 14  tagline                20438 non-null  object 
 15  title  

___
RAFTXO MARCAPÁGINAS 2023-07-05 18:08

In [111]:
data_types = df['production_countries'].apply(type).unique()
if len(data_types) == 1:
    print(f"All data in 'production_countries' column have the same data type: {data_types[0].__name__}")
else:
    print("Data in 'production_countries' column have different data types.")

Data in 'production_countries' column have different data types.


In [112]:
data_types = df['production_countries'].apply(type).unique()
print(data_types)

[<class 'list'> <class 'str'>]


In [113]:
str_data = df[df['production_countries'].apply(lambda x: isinstance(x, str))]
print(str_data['production_countries'])

# 19801 id=82663 no tiene ni título ni casi ningún dato - candidata a eliminar
# 29584 id=122662 es una repe de 'Mardock Scramble: The First Compression' película japonesa - candidata a eliminar
# 35667 id=249260 posiblemente sea una fila con datos perdidos de la película (tv-movie) "Avalanche Sharks" - se puede borrar

19801    
29584    
35667    
Name: production_countries, dtype: object


In [114]:
nan_rows = df[df['title'].isna()]
print(nan_rows)

# coinciden las 3 filas que hemos visto antes al analizar la columna 'production_countries'

           id belongs_to_collection budget  \
19801   82663                            0   
29584  122662      Mardock Scramble      0   
35667  249260                            0   

                                            genres original_language  \
19801                    [Action, Thriller, Drama]                en   
29584                 [Animation, Science Fiction]                ja   
35667  [TV Movie, Action, Horror, Science Fiction]                en   

                                                overview popularity  \
19801  British soldiers force a recently captured IRA...        NaN   
29584         Third film of the Mardock Scramble series.        NaN   
35667  A group of skiers are terrorized during spring...        NaN   

      production_companies production_countries release_date  revenue  \
19801                                                    NaN      NaN   
29584                                                    NaN      NaN   
35667                 

In [115]:
# Procedemos a eliminar esas filas estorbantes:
df = df.dropna(subset=['title'])

In [116]:
has_na = df['title'].isna().any()
print(has_na)

False


In [119]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 45535 entries, 0 to 45537
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     45535 non-null  object 
 1   belongs_to_collection  45535 non-null  object 
 2   budget                 45535 non-null  object 
 3   genres                 45535 non-null  object 
 4   original_language      45524 non-null  object 
 5   overview               44576 non-null  object 
 6   popularity             45535 non-null  object 
 7   production_companies   45535 non-null  object 
 8   production_countries   45535 non-null  object 
 9   release_date           45451 non-null  object 
 10  revenue                45535 non-null  float64
 11  runtime                45278 non-null  float64
 12  spoken_languages       45535 non-null  object 
 13  status                 45454 non-null  object 
 14  tagline                20438 non-null  object 
 15  title  

___
vamos a hacer otro backup

Los siguientes dos líneas de código cambian o intentan cambiar el tipo de datos de la columna 'title'

In [88]:
df['title'] = df['title'].astype(str)

C:\Users\raftx\AppData\Local\Temp\ipykernel_13980\2924549625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].astype(str)


In [90]:
df.loc[:, 'title'] = df['title'].astype(str)

In [103]:
df['production_countries'].tail()

45533                        ["['Iran']"]
45534                 ["['Philippines']"]
45535    ["['United States of America']"]
45536                      ["['Russia']"]
45537              ["['United Kingdom']"]
Name: production_countries, dtype: object

45533                        ["['Iran']"]
45534                 ["['Philippines']"]
45535    ["['United States of America']"]
45536                      ["['Russia']"]
45537              ["['United Kingdom']"]
Name: production_countries, dtype: object

In [104]:
# Verificar si hay valores nulos
null_values = df['production_countries'].isnull().sum()
print("Cantidad de valores nulos:", null_values)


Cantidad de valores nulos: 0


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55646 entries, 0 to 45537
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     55646 non-null  object 
 1   belongs_to_collection  55646 non-null  object 
 2   budget                 55646 non-null  object 
 3   genres                 55646 non-null  object 
 4   original_language      55631 non-null  object 
 5   overview               54575 non-null  object 
 6   popularity             55646 non-null  object 
 7   production_companies   55646 non-null  object 
 8   production_countries   55646 non-null  object 
 9   release_date           55560 non-null  object 
 10  revenue                55646 non-null  float64
 11  runtime                55367 non-null  float64
 12  spoken_languages       55646 non-null  object 
 13  status                 55562 non-null  object 
 14  tagline                25317 non-null  object 
 15  title  

In [ ]:
# Reemplazar valores NaN con un valor vacío
df['production_countries'] = df['production_countries'].fillna('')

In [ ]:
# Desanidar la columna 'production_countries' y extraer solo el valor del nombre
df_production_countries = movies_dataset['production_countries'].apply(lambda x: [country['name'] for country in x] if isinstance(x, list) else [])

In [95]:
filtered_rows = df[df['production_countries'].str.contains('Argentina', case=False)]
print(filtered_rows[['id','production_countries','title','original_language']])

KeyError: "None of [Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n       ...\n       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n      dtype='float64', length=45429)] are in the [columns]"

In [91]:
sharks_rows = df[df['title'].str.contains('sharks', case=False)]
print(sharks_rows)

           id belongs_to_collection budget  \
318     20306                            0   
22541   83896                            0   
31491  259075                            0   
36434  257450                            0   
38486  241004                            0   
39685   46658                            0   

                                        genres original_language  \
318                            [Crime, Comedy]                en   
22541        [Comedy, Horror, Science Fiction]                en   
31491              [Action, Adventure, Horror]                es   
36434                         [Horror, Comedy]                en   
38486  [Adventure, Mystery, Family, Animation]                fr   
39685                  [Horror, Action, Drama]                fr   

                                                overview popularity  \
318    A young, naive Hollywood studio assistant fina...   5.869261   
22541  Just when you thought it was safe to go back t...  

In [75]:
df.iloc[35560:35563]

,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,director_names
35666,89145,The Bowery Boys,0,"[Drama, Comedy, Crime]",en,Slip invites his cousin Jimmy to stay with his...,0.1446,[Monogram Pictures],[United States of America],1948-03-07 00:00:00,0.0,0.0,[English],Released,TWO-FISTED DRAMA!,Angels' Alley,3.0,1.0,[William Beaudine]
35667,249260,,0,"[TV Movie, Action, Horror, Science Fiction]",en,A group of skiers are terrorized during spring...,NaN,,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,[Scott Wheeler]
35668,151911,,0,"[War, Drama]",en,A simple peasant is forced to take up arms to ...,0.69411,"[United Artists, Walter Wanger Productions]",[United States of America],1938-06-17 00:00:00,0.0,85.0,[English],Released,Romance under Fire!,Blockade,0.0,0.0,[William Dieterle]


In [57]:
df['production_countries']

0        [United States of America]
1        [United States of America]
2        [United States of America]
3        [United States of America]
4        [United States of America]
                    ...            
45533                        [Iran]
45534                 [Philippines]
45535    [United States of America]
45536                      [Russia]
45537              [United Kingdom]
Name: production_countries, Length: 45432, dtype: object

In [93]:
# comprobamos la columna 'id'
check_duplicates(df, 'id')

Total Duplicates in id: 0


___

In [52]:
# Eliminar todas las filas duplicadas por el valor de 'id', ya que debe ser único.
df.drop_duplicates(subset=['id', 'title'], keep='first', inplace=True)

In [54]:
# Filter to rows with title 'Cinderella'
congress2 = df[df['production_countries'] == 'United States of America']

# Print title and other relevant columns
print(congress2[['title', 'director_names', 'release_date', 'id']])

Empty DataFrame
Columns: [title, director_names, release_date, id]
Index: []


In [90]:
# Convert lists to strings
df['production_countries'] = df['production_countries'].apply(str)

# Count distinct values   
num_countries = df['production_countries'].nunique()
counts = df['production_countries'].value_counts()

print(num_countries)
print(counts)

2391
production_countries
['United States of America']                        17841
[]                                                   6278
['United Kingdom']                                   2238
['France']                                           1653
['Japan']                                            1354
                                                    ...  
['Finland', 'Germany', 'Netherlands']                   1
['France', 'Denmark', 'Spain', 'Sweden']                1
['France', 'United States of America', 'Canada']        1
['France', 'Monaco', 'Spain', 'Switzerland']            1
['Egypt', 'Italy', 'United States of America']          1
Name: count, Length: 2391, dtype: int64


### aquí hay un problemilla, porque trata ['France'] diferente a ['France','Denmark']
#### no cuenta bien todas las apariciones de 'France'
y lo peor de todo que luego la función de películas por país tampoco rula
dice películas hechas en Francia 0

In [91]:
unique_countries = counts.index

print(unique_countries)

# ['USA', 'UK', 'France', 'China', 'India']

Index(['['United States of America']', '[]', '['United Kingdom']',
       '['France']', '['Japan']', '['Italy']', '['Canada']', '['Germany']',
       '['India']', '['Russia']',
       ...
       '['United States of America', 'Nicaragua', 'France']',
       '['United Kingdom', 'Cayman Islands', 'Sweden']',
       '['France', 'Italy', 'Belgium', 'Japan']',
       '['Australia', 'South Africa']',
       '['Romania', 'United Kingdom', 'Canada']',
       '['Finland', 'Germany', 'Netherlands']',
       '['France', 'Denmark', 'Spain', 'Sweden']',
       '['France', 'United States of America', 'Canada']',
       '['France', 'Monaco', 'Spain', 'Switzerland']',
       '['Egypt', 'Italy', 'United States of America']'],
      dtype='object', name='production_countries', length=2391)


In [72]:
df.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'director_names'],
      dtype='object')

In [73]:
# voy a ver si hay valores repetidos en 'id'
id_counts = df['id'].value_counts()

In [79]:
print(id_counts)

id
862       1
74458     1
296206    1
107308    1
16247     1
         ..
32985     1
44399     1
10138     1
32084     1
461257    1
Name: count, Length: 45432, dtype: int64


In [80]:
df.shape

(45432, 19)

In [75]:
most_common = id_counts.sort_values(ascending=False).head()

In [76]:
print(most_common)

id
862      1
2086     1
1710     1
9087     1
15602    1
Name: count, dtype: int64


___


In [98]:
df.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'director_names'],
      dtype='object')

___

In [100]:
def peliculas_pais(pais):
    global df

    # Explode the lists into separate rows 
    df['production_countries'] = df['production_countries'].apply(str).str.split(',')
    df = df.explode('production_countries')
    
    # Count the number of rows for the given country   
    num_peliculas = df[df['production_countries'] == pais].shape[0]
    
    # Return string with result
    return f"Se produjeron {num_peliculas} peliculas en el pais {pais}"

# Usage example
result = peliculas_pais('France') 
print(result)


Se produjeron 0 peliculas en el pais France


C:\Users\raftx\AppData\Local\Temp\ipykernel_13980\2560518445.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['production_countries'] = df['production_countries'].apply(str).str.split(',')


In [101]:
def peliculas_pais(pais):
    global df
     # Explode the lists into separate rows 
    df.loc[:, 'production_countries'] = df['production_countries'].str.split(',')
    df = df.explode('production_countries')
     # Count the number of rows for the given country   
    num_peliculas = df[df['production_countries'] == pais].shape[0]
     # Return string with result
    return f"Se produjeron {num_peliculas} peliculas en el pais {pais}"
 # Usage example
result = peliculas_pais('France') 
print(result)

Se produjeron 0 peliculas en el pais France


In [82]:
def peliculas_pais(pais):
    '''
    Se ingresa un país, retornando la cantidad de peliculas producidas en el mismo.

    Args:
        pais (str): El nombre del país.

    Returns:
        Un string: 'Se produjeron X películas en el país Y'

    '''

    # filtrar las películas producidas en el país dado
    peliculas_pais = df[df['production_countries'].str.contains(pais, case=False, na=False)]

    # obtener la cantidad de películas producidas en ese país
    cantidad = len(peliculas_pais)

    resultado = f"Se produjeron {cantidad} películas en el país {pais}"

    return resultado

In [83]:
peliculas_pais('United States')

'Se produjeron 0 películas en el país United States'

In [ ]:
peliculas_pais('Argentina')

___


In [ ]:
# @app.get('/get_director')
def get_director(nombre_director):
    retorno=[]
    peliculas=[]
    indice = []

    for index, row in df.iterrows():
        if nombre_director in row["directors"]:
            peliculas.append(row['title'])
            retorno.append(row["return"])
            indice.append(index)
        else:
            continue

    new_retorno = [valor for valor in retorno if valor != 0]
    retorno_final = sum(new_retorno)
    retorno_sum = sum(retorno)

    if len(peliculas) > 1:
        promedio = retorno_final/len(new_retorno)
        peliculas_seleccionadas = df.loc[indice]
        peliculas_part = []
        fecha_lanzamiento = []
        retorno = []
        costo = []
        revenue = []
        for _, fila in peliculas_seleccionadas.iterrows():
            peliculas_part.append(fila['title'])
            fecha_lanzamiento.append(fila['release_date'])
            revenue.append(fila['revenue'])
            costo.append(fila['budget'])
            retorno.append(fila{'return'})
        return {'director':nombre_director, 'retorno_total_director':retorno_final, 'peliculas':peliculas_part, 'anio':fecha_lanzamiento, 'retorno_pelicula':retorno,'budget_pelicula':costo,'revenue_pelicula':revenue}
    else:
        return {'patata':nombre_director}

In [ ]:
# Ejemplo de función que busca las películas de un director:
def get_director_movie_titles(director_name):  
    director_movies = []
    director_rows = None
    try:
       # Filtrar sólo las celdas de directores
       # director_rows = [df['director_names']]
       
       # Encontrar el director específico  
       director_row = director_rows[director_rows['crew'].apply(lambda x: json.loads(x)['name'] == director_name)] 
       
       # Obtener el ID de crédito  
       credit_id = director_row['credit_id'].values[0]  
  
       # Encontrar las películas con ese ID
       movie_rows = df_movies[df_movies['credit_id'] == credit_id]  
  
       # Extraer los títulos de películas  
       titles = movie_rows['title'].tolist()
       director_movies.extend(titles)
        
    except JSONDecodeError:
       pass  
        
    # Buscar cualquier otra fila con el mismo director        
    for row in director_rows['crew']:
       try:
           crew = json.loads(row)
           if crew['name'] == director_name:  
               credit_id = crew['credit_id']  
               movie_rows = df_movies[df_movies['credit_id'] == credit_id]  
               titles = movie_rows['title'].tolist()
               director_movies.extend(titles)  
       except JSONDecodeError:
           pass
           
    return f"{director_name} directed: {', '.join(director_movies)}"

___  
## DIRECTOR

In [ ]:
# Comprobar el funcionamiento de la función get_director...
spielberg_movies = get_director_movie_titles("Steven Spielberg")
print(spielberg_movies)
# ['Jaws', 'Raiders of the Lost Ark', 'E.T. the Extra-Terrestrial', ...]

Dado el éxito del tratamiento de la columna `belongs_to_collection` vamos a seguir transformando las otras columnas de este tipo

In [ ]:
# comprobar los tipos de la columna 'genres'
# Ver los tipos únicos 
df_movies['genres'].apply(type).unique()
# al devolver solo un tipo (string), no la vamos a tocar más.


In [ ]:
# un momento, en vez de ir comprobando una por una, vamos a hacerlo a la vez
for col in df_movies.columns:
    col_types = df_movies[col].apply(type).unique()
    print(f"{col}: {col_types}")

___  
### Transformaciones de la columna `budget`

In [ ]:
# Hemos identificado que hay otras columnas con disparidad de tipos...
# Vamos a por la primera: 'budget'
# Filtrar solo strings
strings = df_movies[df_movies['budget'].apply(type) == str]['budget']

print(strings)

In [ ]:
int_count = df_movies['budget'].apply(lambda x: 1 if type(x)==int else 0).sum()
str_count = df_movies['budget'].apply(lambda x: 1 if type(x)==str else 0).sum()

print(f"Number of int values: {int_count}") 
print(f"Number of str values: {str_count}")

In [ ]:
# antes hemos identificado 3 strings en la variable strings
# así que podemos sustituir lo que sea que contuvieran por 0 (integer directamente)
df_movies.loc[strings.index, 'budget'] = 0

In [ ]:
# vamos a por los nans del 'budget'
nan_values = df_movies['budget'].isna()


In [ ]:
print(nan_values) # no parece que haya ningún nan

In [ ]:
# a ver cuántos 'budget' hay distintos a 0
print(df_movies[df_movies['budget']!=0]['budget'])

In [ ]:
# aunque en la API no se usa la columna 'budget' en las transformaciones se requiere
# que los nulos sean 0 (y también hemos visto que hay 3 strings - los pondremos a 0 igualmente)
df_movies.loc[nan_values, 'budget'] = 0

In [ ]:
print(df_movies[df_movies['budget']==0]['budget'])

___  
### Transformaciones `revenue`

In [ ]:
nan_values = df_movies['revenue'].isna()
print(nan_values)
# reemplazar los valores NaN en la columna 'revenue' con el valor 0
df_movies.loc[nan_values, 'revenue'] = 0


In [ ]:
# comprobar cuántos 'revenue' hay iguales a 0
print(df_movies[df_movies['revenue']==0]['revenue'])
# y cuántos distintos a 0
print(df_movies[df_movies['revenue']!=0]['revenue'])

# conclusión: hay muy pocos datos en 'revenue', la mayoría eran nulos y ahora son 0

___

In [ ]:
df_movies.columns

In [ ]:
# comprobar la columna 'original_language'
df_movies['original_language'].apply(type).unique()

In [ ]:
# Ver tipos únicos
types = df_movies['original_language'].apply(type).unique()

# Clasificar valores por tipo
strings = df_movies[df_movies['original_language'].apply(type) == str]['original_language']
floats = df_movies[df_movies['original_language'].apply(type) == float]['original_language']  
integers = df_movies[df_movies['original_language'].apply(type) == int]['original_language']

# Imprimir valores clasificados
print(f"Strings: {strings}")
print(f"Floats: {floats}") 
print(f"Integers: {integers}")

In [ ]:
# compruebo qué películas son esas de 'original_language' igual a NaN
float_nans = df_movies[df_movies['original_language'].apply(type) == float]

float_nans[['original_language', 'title','release_date']]

In [ ]:
# Convertir a string
df_movies['original_language'] = df_movies['original_language'].astype(str)

In [ ]:
# comprobar nuevamente la columna 'original_language' tras la conversión
df_movies['original_language'].apply(type).unique()

In [ ]:
# comprobar qué filas contienen en la columna 'original_language' un valor 
# más largo de 2 caracteres

mask = df_movies['original_language'].str.len() > 2
df_movies[mask]

In [ ]:
# Identificar valores a reemplazar de la columna 'original_language'
condition1 = df_movies['original_language'] == '104'
condition2 = df_movies['original_language'] == '68' 
condition3 = df_movies['original_language'] == '82'
condition4 = df_movies['original_language'] == 'nan'

condition = condition1 | condition2 | condition3 | condition4

# Reemplazar esos valores con 'unknown'
df_movies.loc[condition, 'original_language'] = 'unknown' 